### Introduction
In this code, we calculated the intersection percentage between PRIO GRID cell which has a SNL mining and PRIO GRID cell code which has indigenous land on it.

However, we add a 3*3 buffer for each PRIO GRID cell where there is any indigenous land on it

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd


### Read PRIO GRID data

In [2]:
gdf_priogrid_tmp = gpd.read_file("../input/priogrid_cellshp/priogrid_cell.shp")[["col","row","gid"]]
gdf_priogrid_tmp.dtypes


col    int64
row    int64
gid    int64
dtype: object

In [3]:
print(gdf_priogrid_tmp.shape)

(259200, 3)


In [4]:
# build a dictionary where store the row number and column number of each prio grid cell
dict={}
def dictionary_func(col, row, id):
    dict[(row,col)]=id   

In [5]:
gdf_priogrid_tmp[["col","row","gid"]].apply(lambda x: dictionary_func(*x), axis=1)
dict

{(360, 688): 259168,
 (360, 689): 259169,
 (360, 690): 259170,
 (360, 691): 259171,
 (360, 692): 259172,
 (360, 693): 259173,
 (360, 694): 259174,
 (360, 695): 259175,
 (360, 696): 259176,
 (360, 697): 259177,
 (360, 698): 259178,
 (360, 699): 259179,
 (360, 700): 259180,
 (360, 701): 259181,
 (360, 702): 259182,
 (360, 703): 259183,
 (360, 704): 259184,
 (360, 705): 259185,
 (360, 706): 259186,
 (360, 707): 259187,
 (360, 708): 259188,
 (360, 709): 259189,
 (360, 710): 259190,
 (360, 711): 259191,
 (360, 712): 259192,
 (360, 713): 259193,
 (360, 714): 259194,
 (360, 715): 259195,
 (360, 716): 259196,
 (360, 717): 259197,
 (360, 718): 259198,
 (360, 719): 259199,
 (360, 720): 259200,
 (360, 648): 259128,
 (360, 649): 259129,
 (360, 650): 259130,
 (360, 651): 259131,
 (360, 652): 259132,
 (360, 653): 259133,
 (360, 654): 259134,
 (360, 655): 259135,
 (360, 656): 259136,
 (360, 657): 259137,
 (360, 658): 259138,
 (360, 659): 259139,
 (360, 660): 259140,
 (360, 661): 259141,
 (360, 662): 

In [6]:
gdf_priogrid = gpd.read_file("../input/priogrid_cellshp/priogrid_cell.shp")
gdf_priogrid.head()

gid  xcoord  ycoord  col  row  \
0  259168  163.75   89.75  688  360   
1  259169  164.25   89.75  689  360   
2  259170  164.75   89.75  690  360   
3  259171  165.25   89.75  691  360   
4  259172  165.75   89.75  692  360   

                                            geometry  
0  POLYGON ((163.50000 89.50000, 163.50000 90.000...  
1  POLYGON ((164.00000 89.50000, 164.00000 90.000...  
2  POLYGON ((164.50000 89.50000, 164.50000 90.000...  
3  POLYGON ((165.00000 89.50000, 165.00000 90.000...  
4  POLYGON ((165.50000 89.50000, 165.50000 90.000...

### Read indigenous land data

In [7]:
ind_land=pd.read_csv("../input/ind_land.csv",usecols=['GRID IDs'])
print(ind_land.shape)

(159050, 1)


/var/folders/0p/mkzn2l513fl51bl6rs96d9z40000gn/T/ipykernel_18667/3232504066.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  ind_land=pd.read_csv("../input/ind_land.csv",usecols=['GRID IDs'])


In [8]:
ind_land= ind_land[pd.to_numeric(ind_land['GRID IDs'], errors='coerce').notnull()]
ind_land["GRID IDs"] = ind_land["GRID IDs"].astype("int")
print(ind_land.shape)

(156873, 1)


In [9]:
priogrid_ind_land = ind_land.merge(gdf_priogrid, how='left', left_on='GRID IDs',right_on='gid') 
print(priogrid_ind_land.shape)
priogrid_ind_land.head()


(156873, 7)


GRID IDs     gid  xcoord  ycoord  col  row  \
0     68612   68612  -74.25  -42.25  212   96   
1    157122  157122  -99.25   19.25  162  219   
2    157122  157122  -99.25   19.25  162  219   
3    157122  157122  -99.25   19.25  162  219   
4    157122  157122  -99.25   19.25  162  219   

                                            geometry  
0  POLYGON ((-74.50000 -42.50000, -74.50000 -42.0...  
1  POLYGON ((-99.50000 19.00000, -99.50000 19.500...  
2  POLYGON ((-99.50000 19.00000, -99.50000 19.500...  
3  POLYGON ((-99.50000 19.00000, -99.50000 19.500...  
4  POLYGON ((-99.50000 19.00000, -99.50000 19.500...

###  Create 3*3 Buffer for each PRIO GRID cell with indigenous land on it

In [10]:
ls_each_row= []
ls_total=[]
def buffer_prio_id(x,y,row, col):
    buffer_id=[]
    # there are 4 special case
    # where lat = -89.75/89.75
    # long = -179.75/179.75
    if y == -89.75:
        l_row=[row, row+1]

    elif y == 89.75:
        l_row=[row-1, row]
    else:
        l_row=[row-1, row, row+1]

    if x == -179.75:
        l_col=[720, col, col+1]

    elif x == 179.75:
        l_col=[col-1, col, 1]

    else:
        l_col=[col-1, col, col+1]  
   
    for i in l_row:
        for j in l_col:
            tmp=dict[(int(i), int(j))]
            buffer_id.append(tmp)
            ls_total.append(tmp)
    ls_each_row.append(buffer_id)        
    return buffer_id

In [11]:
priogrid_ind_land["buffer_id"]= priogrid_ind_land[["xcoord", "ycoord","row","col"]].apply(lambda x: buffer_prio_id(*x), axis=1)

In [12]:
ls_total = set(ls_total)

In [13]:
priogrid_ind_land.to_csv("../output/priogrid_ind_land_With_buffer_id.csv")

### Read SNL data

In [14]:
df_SNL= pd.read_csv("../input/SNL-PRIO GRID IDs.csv")
print(df_SNL.shape)
df_SNL.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
df_SNL = df_SNL[pd.to_numeric(df_SNL['PRIO GRID ID'], errors='coerce').notnull()]
df_SNL["PRIO GRID ID"] = df_SNL["PRIO GRID ID"].astype("int")
df_WHC = df_SNL[(df_SNL['LATITUDE']<=90) & (df_SNL['LATITUDE']>= -90) &
                (df_SNL['LONGITUDE']<=180) & (df_SNL['LONGITUDE']>= -180)  ].reset_index().drop(columns="index",axis=1)
print(df_SNL.shape)
df_SNL.head()                

(39612, 46)
(39370, 46)


/var/folders/0p/mkzn2l513fl51bl6rs96d9z40000gn/T/ipykernel_18667/2781643676.py:1: DtypeWarning: Columns (2,22,23,24,25,26,27,28,30,31,32,33,36,37,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_SNL= pd.read_csv("../input/SNL-PRIO GRID IDs.csv")


Unnamed: 0           Property name Property ID Also Known As  \
0          0           11 RAU-Novaya       68289           NaN   
1          1           12 Mile Creek       66178           NaN   
2          2               14 Karats       65525           NaN   
3          3  14 Mile Well Extension       63665           NaN   
4          4     15 Mile Silver City       65760           NaN   

                                     OWNER(S) Equity ownership %  \
0  Sodeistvie Private Enterprise (Owner) 100%                100   
1               U&D Coal Limited (Owner) 100%                100   
2          Dios Exploration Inc. (Owner) 100%                100   
3               Private Interest (Owner) 100%                100   
4       North Lily Mining Company (Owner) 50%                 50   

  DEVELOPMENT STAGE ACTIVITY STATUS   COMMODITY(S) PRIMARY COMMODITY  ...  \
0        Grassroots        Inactive           Coal              Coal  ...   
1        Grassroots        Inactive           Coal              Coal  ...   
2        Grassroots        Inactive  Gold,Diamonds              Gold  ...   
3        Grassroots        Inactive           Gold              Gold  ...   
4        Grassroots        Inactive    Gold,Silver              Gold  ...   

  Unnamed: 36 Unnamed: 37 Unnamed: 38 Unnamed: 39 Unnamed: 40  Unnamed: 41  \
0         NaN         NaN         NaN         NaN         NaN          NaN   
1         NaN         NaN         NaN         NaN         NaN          NaN   
2         NaN         NaN         NaN         NaN         NaN          NaN   
3         NaN         NaN         NaN         NaN         NaN          NaN   
4         NaN         NaN         NaN         NaN         NaN          NaN   

   Unnamed: 42 Unnamed: 43 Unnamed: 44 PRIO GRID ID  
0          NaN         NaN         NaN       199878  
1          NaN         NaN         NaN        96419  
2          NaN         NaN         NaN       205418  
3          NaN         NaN         NaN        88445  
4          NaN         NaN         NaN       221841  

[5 rows x 46 columns]

In [15]:
list_SNL= set(df_SNL["PRIO GRID ID"].tolist())

### Find the overlapping between PRIO GRID of SNL mining and indigenous Land 

There are 4 ways of calculation. The main differences are:

1. if we are going to iterate through each SNL mining or Indigenous land first;

2. and the denominator is the total number SNL mining or total number of indigenous land


Find the intersection PRIO GRID IDs by the first way: iterate through the SNL first

```
count = 0 
 Loop through PRIO GRID IDs of SNL mining: 
    Loop thourhg the PRIO GRID IDs of indigenous land in ls_total:
        if there is PRIO GRID IDs where has indigenous land and SNL data mining:
            count +=1 
```

In [16]:
tmp_1 = []
def check_id(x): #iterate every SNL project
    if int(x) in ls_total: #if the grid cell of the SNL project is located at any indigenousgeous land buffer
        tmp_1.append(x) #count+=1


In [17]:
df_SNL['PRIO GRID ID'].apply(lambda x: check_id(x))

0        None
1        None
2        None
3        None
4        None
         ... 
39607    None
39608    None
39609    None
39610    None
39611    None
Name: PRIO GRID ID, Length: 39370, dtype: object

Find the intersection PRIO GRID IDs by the second way: iterate through the indigenous land first
```
count = 0 
Loop thourhg the PRIO GRID IDs of indigenous lands:
    Loop through PRIO GRID IDs of SNL mining in list_SNL:
        if there is PRIO GRID IDs where has indigenous land and SNL data mining:
            count +=1 
```

In [18]:
tmp_2=[]
def check_id_2(x): #iterate through each indigenous land
    for j in x:#iterate through each grid cell id in one indigenous land's buffer
        if j in list_SNL: #there is any buffer cell located at the same cell as WB project
            tmp_2.append(j) #counter+=1
            break

In [19]:
priogrid_ind_land['buffer_id'].apply(lambda x: check_id_2(x))

0         None
1         None
2         None
3         None
4         None
          ... 
156868    None
156869    None
156870    None
156871    None
156872    None
Name: buffer_id, Length: 156873, dtype: object

Calculate the percentage of overlapping between PRIO GRID cell of SNL mining and indigenous Land = count / total SNL dataset

The interception percentage calculated by iterating though the each SNL first is : 26.81%

The interception percentage calculated by iterating though the each indigenous land is : 361.28%

In [20]:
print(len(tmp_1)/df_SNL.shape[0])
print(len(tmp_2)/df_SNL.shape[0])

0.2681737363474727
3.612801625603251


Calculate the percentage of overlapping between PRIO GRID cell of SNL mining and indigenous Land = count / total indigenous land number

The interception percentage calculated by iterating though the each SNL is :6.73%

The interception percentage calculated by iterating though the each indigenous land is :90.66%

In [21]:
print(len(tmp_1)/priogrid_ind_land.shape[0])
print(len(tmp_2)/priogrid_ind_land.shape[0])

0.06730285007617627
0.9066952247996787
